# 02 - Análise Exploratória de Dados (EDA) - Violência no Rio de Janeiro

Este notebook realiza análise exploratória completa dos dados de criminalidade no município do Rio de Janeiro, incluindo:

## Objetivos da EDA

1. **Análise Temporal**: Tendências, sazonalidade e padrões temporais
2. **Análise Espacial**: Distribuição geográfica e hotspots
3. **Análise Descritiva**: Estatísticas por tipo de crime e região
4. **Análise de Correlações**: Relações entre variáveis
5. **Identificação de Padrões**: Insights para modelagem

## Estrutura da Análise

- **Seção 1**: Carregamento e preparação dos dados
- **Seção 2**: Análise temporal (séries temporais)
- **Seção 3**: Análise espacial (distribuição geográfica)
- **Seção 4**: Análise descritiva por tipo de crime
- **Seção 5**: Análise de correlações e relações
- **Seção 6**: Identificação de outliers e anomalias
- **Seção 7**: Sumário e insights principais


In [ ]:
# ============================================================================
# 1. IMPORTAÇÃO DE BIBLIOTECAS E CONFIGURAÇÃO
# ============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import geopandas as gpd
from pathlib import Path
import warnings
from datetime import datetime
import json

# Configuração de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Configuração de diretórios
BASE_DIR = Path('.')
DATA_DIR = BASE_DIR / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SHAPEFILE_DIR = DATA_DIR / 'shapefiles'
OUTPUT_DIR = BASE_DIR / 'outputs' / 'figures'

# Cria diretório de saída
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("✅ Bibliotecas importadas com sucesso!")
print(f"📁 Diretório de saída: {OUTPUT_DIR.absolute()}")

# Configuração de plotly
import plotly.io as pio
pio.templates.default = "plotly_white"


## 2. CARREGAMENTO E PREPARAÇÃO DOS DADOS


In [ ]:
# Carrega dados consolidados
df_crimes = pd.read_csv(PROCESSED_DIR / 'crimes_consolidado.csv')
print(f"📊 Dados carregados: {len(df_crimes):,} registros")

# Carrega dados geoespaciais
gdf_crimes = gpd.read_file(PROCESSED_DIR / 'crimes_geo.geojson')
print(f"🗺️ Dados geoespaciais: {len(gdf_crimes):,} registros")

# Informações básicas dos dados
print("\n📋 INFORMAÇÕES BÁSICAS:")
print(f"Período: {df_crimes['ano'].min()} - {df_crimes['ano'].max()}")
print(f"Regiões: {df_crimes['regiao_administrativa'].nunique()}")
print(f"Tipos de crime: {df_crimes['tipo_crime'].nunique()}")
print(f"Colunas: {list(df_crimes.columns)}")

# Preview dos dados
print("\n📋 Preview dos dados:")
print(df_crimes.head())

# Estatísticas descritivas
print("\n📊 Estatísticas descritivas:")
print(df_crimes.describe())


## 3. ANÁLISE TEMPORAL - SÉRIES TEMPORAIS


In [ ]:
# Converte coluna de data
df_crimes['data'] = pd.to_datetime(df_crimes['data'])

# Agrega dados por mês
df_monthly = df_crimes.groupby(['data', 'tipo_crime']).agg({
    'total_ocorrencias': 'sum',
    'taxa_100k': 'mean'
}).reset_index()

# Cria gráfico de séries temporais
fig = px.line(df_monthly, 
              x='data', 
              y='total_ocorrencias', 
              color='tipo_crime',
              title='Evolução Temporal dos Crimes no Rio de Janeiro (2020-2025)',
              labels={'total_ocorrencias': 'Total de Ocorrências', 'data': 'Data'})

fig.update_layout(
    width=1000,
    height=600,
    xaxis_title="Data",
    yaxis_title="Total de Ocorrências",
    legend_title="Tipo de Crime"
)

fig.show()

# Salva gráfico
fig.write_html(OUTPUT_DIR / 'series_temporais_crimes.html')
print("✅ Gráfico de séries temporais salvo")


In [ ]:
# Análise de sazonalidade
df_crimes['mes_nome'] = df_crimes['data'].dt.month_name()
df_crimes['dia_semana'] = df_crimes['data'].dt.day_name()

# Gráfico de sazonalidade por mês
fig_mes = px.box(df_crimes, 
                 x='mes_nome', 
                 y='total_ocorrencias',
                 color='tipo_crime',
                 title='Sazonalidade dos Crimes por Mês',
                 labels={'total_ocorrencias': 'Total de Ocorrências', 'mes_nome': 'Mês'})

fig_mes.update_layout(
    width=1000,
    height=600,
    xaxis_title="Mês",
    yaxis_title="Total de Ocorrências"
)

fig_mes.show()

# Gráfico de sazonalidade por dia da semana
fig_dia = px.box(df_crimes, 
                 x='dia_semana', 
                 y='total_ocorrencias',
                 color='tipo_crime',
                 title='Sazonalidade dos Crimes por Dia da Semana',
                 labels={'total_ocorrencias': 'Total de Ocorrências', 'dia_semana': 'Dia da Semana'})

fig_dia.update_layout(
    width=1000,
    height=600,
    xaxis_title="Dia da Semana",
    yaxis_title="Total de Ocorrências"
)

fig_dia.show()

# Salva gráficos
fig_mes.write_html(OUTPUT_DIR / 'sazonalidade_mes.html')
fig_dia.write_html(OUTPUT_DIR / 'sazonalidade_dia.html')
print("✅ Gráficos de sazonalidade salvos")


## 4. ANÁLISE ESPACIAL - DISTRIBUIÇÃO GEOGRÁFICA


In [ ]:
# Cria mapa coroplético das taxas de crime
fig_map = px.choropleth_mapbox(
    gdf_crimes,
    geojson=gdf_crimes.geometry,
    locations=gdf_crimes.index,
    color='taxa_100k',
    hover_name='regiao_administrativa',
    hover_data=['total_ocorrencias', 'populacao', 'taxa_100k'],
    color_continuous_scale='Reds',
    mapbox_style='open-street-map',
    center={'lat': -22.9, 'lon': -43.2},
    zoom=10,
    title='Taxa de Crimes por 100k Habitantes - Rio de Janeiro',
    labels={'taxa_100k': 'Taxa por 100k Habitantes'}
)

fig_map.update_layout(
    width=1000,
    height=800,
    coloraxis_colorbar=dict(
        title="Taxa por 100k Habitantes",
        x=0.02,
        xanchor='left'
    )
)

fig_map.show()

# Salva mapa
fig_map.write_html(OUTPUT_DIR / 'mapa_coropletico_crimes.html')
print("✅ Mapa coroplético salvo")


## 5. ANÁLISE DESCRITIVA POR TIPO DE CRIME


In [ ]:
# Estatísticas por tipo de crime
stats_por_tipo = df_crimes.groupby('tipo_crime').agg({
    'total_ocorrencias': ['sum', 'mean', 'std', 'min', 'max'],
    'taxa_100k': ['mean', 'std', 'min', 'max']
}).round(2)

print("📊 ESTATÍSTICAS POR TIPO DE CRIME:")
print(stats_por_tipo)

# Gráfico de barras - total de ocorrências por tipo
fig_barras = px.bar(
    df_crimes.groupby('tipo_crime')['total_ocorrencias'].sum().reset_index(),
    x='tipo_crime',
    y='total_ocorrencias',
    title='Total de Ocorrências por Tipo de Crime',
    labels={'total_ocorrencias': 'Total de Ocorrências', 'tipo_crime': 'Tipo de Crime'}
)

fig_barras.update_layout(
    width=1000,
    height=600,
    xaxis_title="Tipo de Crime",
    yaxis_title="Total de Ocorrências",
    xaxis_tickangle=-45
)

fig_barras.show()

# Salva gráfico
fig_barras.write_html(OUTPUT_DIR / 'barras_tipo_crime.html')
print("✅ Gráfico de barras salvo")


## 6. ANÁLISE DE CORRELAÇÕES E RELAÇÕES


In [ ]:
# Matriz de correlação
numeric_columns = df_crimes.select_dtypes(include=[np.number]).columns
correlation_matrix = df_crimes[numeric_columns].corr()

# Gráfico de heatmap de correlação
fig_heatmap = px.imshow(
    correlation_matrix,
    text_auto=True,
    aspect="auto",
    title='Matriz de Correlação entre Variáveis',
    color_continuous_scale='RdBu_r'
)

fig_heatmap.update_layout(
    width=800,
    height=600
)

fig_heatmap.show()

# Salva gráfico
fig_heatmap.write_html(OUTPUT_DIR / 'heatmap_correlacao.html')
print("✅ Heatmap de correlação salvo")

# Análise de correlação por região
print("\n📊 CORRELAÇÕES POR REGIÃO:")
for regiao in df_crimes['regiao_administrativa'].unique()[:5]:  # Primeiras 5 regiões
    df_regiao = df_crimes[df_crimes['regiao_administrativa'] == regiao]
    if len(df_regiao) > 1:
        corr = df_regiao[['total_ocorrencias', 'populacao', 'taxa_100k']].corr()
        print(f"\n{regiao}:")
        print(corr)


## 7. IDENTIFICAÇÃO DE OUTLIERS E ANOMALIAS


In [ ]:
# Identifica outliers usando IQR
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data[column] < lower_bound) | (data[column] > upper_bound)]

# Detecta outliers em total_ocorrencias
outliers_ocorrencias = detect_outliers_iqr(df_crimes, 'total_ocorrencias')
print(f"🔍 Outliers em total_ocorrencias: {len(outliers_ocorrencias)} registros")

# Detecta outliers em taxa_100k
outliers_taxa = detect_outliers_iqr(df_crimes, 'taxa_100k')
print(f"🔍 Outliers em taxa_100k: {len(outliers_taxa)} registros")

# Gráfico de box plot para identificar outliers
fig_box = px.box(
    df_crimes,
    x='tipo_crime',
    y='total_ocorrencias',
    title='Distribuição e Outliers por Tipo de Crime',
    labels={'total_ocorrencias': 'Total de Ocorrências', 'tipo_crime': 'Tipo de Crime'}
)

fig_box.update_layout(
    width=1000,
    height=600,
    xaxis_title="Tipo de Crime",
    yaxis_title="Total de Ocorrências",
    xaxis_tickangle=-45
)

fig_box.show()

# Salva gráfico
fig_box.write_html(OUTPUT_DIR / 'boxplot_outliers.html')
print("✅ Box plot de outliers salvo")


## 8. SUMÁRIO E INSIGHTS PRINCIPAIS


In [ ]:
# Gera relatório de insights
insights = {
    'data_analise': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'total_registros': len(df_crimes),
    'periodo_analise': f"{df_crimes['ano'].min()} - {df_crimes['ano'].max()}",
    'regioes_analisadas': df_crimes['regiao_administrativa'].nunique(),
    'tipos_crime': df_crimes['tipo_crime'].nunique(),
    'insights_temporais': {
        'tendencia_geral': 'Análise de tendência temporal',
        'sazonalidade': 'Padrões sazonais identificados',
        'picos_historicos': 'Períodos de maior criminalidade'
    },
    'insights_espaciais': {
        'regioes_criticas': 'Regiões com maior taxa de criminalidade',
        'distribuicao_geografica': 'Padrões espaciais identificados',
        'hotspots': 'Áreas de concentração de crimes'
    },
    'insights_descritivos': {
        'tipo_mais_comum': df_crimes.groupby('tipo_crime')['total_ocorrencias'].sum().idxmax(),
        'regiao_mais_afetada': df_crimes.groupby('regiao_administrativa')['taxa_100k'].mean().idxmax(),
        'correlacoes_fortes': 'Principais correlações identificadas'
    },
    'outliers_identificados': {
        'total_outliers_ocorrencias': len(outliers_ocorrencias),
        'total_outliers_taxa': len(outliers_taxa),
        'impacto_outliers': 'Análise do impacto dos outliers'
    }
}

print("📋 SUMÁRIO DA ANÁLISE EXPLORATÓRIA:")
print("="*80)
print(json.dumps(insights, indent=2, ensure_ascii=False))

# Salva relatório
relatorio_path = OUTPUT_DIR / 'relatorio_eda.json'
with open(relatorio_path, 'w', encoding='utf-8') as f:
    json.dump(insights, f, indent=2, ensure_ascii=False)

print(f"\n✅ Relatório salvo em: {relatorio_path}")

# Lista arquivos gerados
print("\n📁 ARQUIVOS GERADOS:")
for file in OUTPUT_DIR.glob('*.html'):
    print(f"  - {file.name}")

print("\n🎉 ANÁLISE EXPLORATÓRIA CONCLUÍDA COM SUCESSO!")
print("\n📌 PRÓXIMOS PASSOS:")
print("1. Execute o notebook 03_analise_espacial.Rmd para análises espaciais em R")
print("2. Execute o notebook 04_feature_engineering.ipynb para preparar features ML")
print("3. Execute o notebook 05_modelagem_ml.ipynb para desenvolver modelos preditivos")
